# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: roBERTa
* Evaluation approach: epoch
* Fine-tuning dataset: imdb

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [4]:
from datasets import load_dataset
import pandas as pd
import numpy as np

splits=["train","test"]
dataset = load_dataset("imdb", split=splits)
dataset = [dataset.rename_column('label', 'labels') for dataset in dataset]
# Crează un dicționar cu seturile de date
ds = {split: dataset.shuffle(seed=42).select(range(500)) for split, dataset in zip(splits, dataset)}

#display(ds)
display(dataset)
print("\n")
display(ds)

[Dataset({
     features: ['text', 'labels'],
     num_rows: 25000
 }),
 Dataset({
     features: ['text', 'labels'],
     num_rows: 25000
 })]

{'train': Dataset({
     features: ['text', 'labels'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['text', 'labels'],
     num_rows: 500
 })}

In [5]:
# here i just vizualize the data to understand it

print("tipul lui ds:", type(ds))
#print(dataset.shape)
print("tipul lui dataset:", type(dataset))
print(dataset,"\n")
display(dataset[0][0]) #for i in range(0,2)]
display(ds["train"][0])

tipul lui ds: <class 'dict'>
tipul lui dataset: <class 'list'>
[Dataset({
    features: ['text', 'labels'],
    num_rows: 25000
}), Dataset({
    features: ['text', 'labels'],
    num_rows: 25000
})] 



{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...',
 'labels': 1}

In [6]:
#from transformers import AutoTokenizer
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import BitsAndBytesConfig
# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length')

tokenized_ds={}
for split in splits:
    tokenized_ds[split] = ds[split].map(tokenize_function, batched=True)
 
    
# Show the first example of the tokenized training set
print(tokenized_ds)
print(tokenized_ds['train'][0])

{'train': Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 500
}), 'test': Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 500
})}
{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'labels': 1, 'i

In [7]:
#from transformers import AutoModelForCausalLM6
import torch

model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=2
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}


training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=1e-4,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    fp16=True,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)


# Inițializează trainer-ul
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


# freeze all paramters
for param in model.base_model.parameters():
   param.requires_grad = True

display(model)


device =  cuda


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\ProgramData\Anaconda3\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [8]:
print("Trainer va folosi device:", training_args.device)
print("Modelul este pe device:", next(model.parameters()).device)

trainer.train()
results = trainer.evaluate()
print(results)


Trainer va folosi device: cuda:0
Modelul este pe device: cuda:0


C:\ProgramData\Anaconda3\Lib\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss,Accuracy
0,0.682500,0.416969,0.836000
1,0.239400,0.270257,0.898000
2,0.123100,0.275931,0.914000
3,0.043300,0.401548,0.902000
4,0.052400,0.431757,0.912000


{'eval_loss': 0.27025726437568665, 'eval_accuracy': 0.898, 'eval_runtime': 5.7027, 'eval_samples_per_second': 87.677, 'eval_steps_per_second': 21.919, 'epoch': 4.8}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [12]:
from peft import get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    task_type = TaskType.SEQ_CLS,
    inference_mode = False,
    r = 8,
    lora_alpha=16,
    lora_dropout=0.1
)

In [14]:
lora_model = get_peft_model(model, peft_config)
lora_model.print_trainable_parameters()

trainable params: 887,042 || all params: 125,534,212 || trainable%: 0.7066


In [15]:
# Inițializează trainer-ul
lora_trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

lora_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,0.168300,0.278541,0.898000
1,0.103000,0.271781,0.904000
2,0.094100,0.297312,0.892000
3,0.072600,0.282215,0.902000
4,0.105600,0.283176,0.902000


TrainOutput(global_step=75, training_loss=0.11738499402999877, metrics={'train_runtime': 118.7226, 'train_samples_per_second': 21.057, 'train_steps_per_second': 0.632, 'total_flos': 638006516121600.0, 'train_loss': 0.11738499402999877, 'epoch': 4.8})

In [17]:
lora_results = lora_trainer.evaluate()
print(lora_results)

{'eval_loss': 0.27178093791007996, 'eval_accuracy': 0.904, 'eval_runtime': 6.4978, 'eval_samples_per_second': 76.949, 'eval_steps_per_second': 19.237, 'epoch': 4.8}


###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [19]:
# Saving the model
#model.save("/tmp/your_model_name")
lora_model.save_pretrained("./lora_model")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [24]:
from peft import AutoPeftModelForSequenceClassification

# Încarcă modelul LoRA salvat
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("./lora_model")



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:

print("Base Model Accuracy:", results["eval_accuracy"])
print("LoRA Model Accuracy:", lora_results["eval_accuracy"])


Base Model Accuracy: 0.898
LoRA Model Accuracy: 0.904
